In [3]:
import pickle
import graphlab as gl
import pandas as pd
from datetime import datetime, timedelta



In [4]:
data=pickle.load(open("/Users/ozad/Desktop/axa_data_challenge-master/callsData_DataFrame.obj","rb"))

In [18]:
print data.head()

   DAY_OFF  WEEK_END  DAY_WE_DS  TPER_TEAM  ASS_ASSIGNMENT  ASS_DIRECTORSHIP  \
0        0         1          6          1              27                 4   
1        0         1          6          1              27                 4   
2        0         1          6          1              27                 4   
3        0         1          6          1              27                 4   
4        0         1          6          1              27                 4   

   ASS_PARTNER  ASS_POLE  CSPL_CALLSOFFERED  CSPL_NOANSREDIR   ...    \
0           15         9                0.0              0.0   ...     
1           15         9                0.0              0.0   ...     
2           15         9                0.0              0.0   ...     
3           15         9                0.0              0.0   ...     
4           15         9                1.0              0.0   ...     

   CSPL_ACCEPTABLE  CSPL_MAXSTAFFED  CSPL_ABANDONNED_CALLS  CSPL_CALLS  \
0           

In [5]:
import numpy as np

class FeatureExtractorClf(object):
    def __init__(self):
        pass

    def fit(self, X_df, y_df):
        pass
    
    def transform(self, X_df):

        XX = np.array([np.array(dd) for dd in X_df['ASS_ASSIGNMENT']])
        return XX

In [16]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.base import BaseEstimator
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.decomposition import PCA, KernelPCA
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis



class Classifier(BaseEstimator):
    def __init__(self):
        self.n_components = 8
        self.n_estimators = 400
        self.clf = Pipeline([
            #('pca', PCA(n_components=self.n_components)), 
            #('pca',  KernelPCA(n_components=21,kernel="rbf",remove_zero_eig=True,gamma=0.0004)),
            
            ####### Basic algo :  Random Forest Classifier
            #('clf', RandomForestClassifier(n_estimators=self.n_estimators, random_state=42))
            
            ####### GradientBoost
            #('clf', GradientBoostingClassifier(n_estimators=self.n_estimators,max_depth=1000,subsample=0.85,
            #       max_features='log2',random_state=42))
                    
                 
           ####### AdaBoost avec DecisionTreeClassifier
            ('clf', AdaBoostClassifier(
                   base_estimator=DecisionTreeClassifier(max_depth=8),
                   n_estimators=100,
                   learning_rate=1.6,
                   random_state=42,
                   algorithm='SAMME',
               ))
                
            #('clf', QuadraticDiscriminantAnalysis()),
            #('clf', LinearDiscriminantAnalysis(solver='svd', shrinkage=None, priors=None, n_components=10, store_covariance=True, tol=0.01))
            
                
                
            ####### AdaBoost avec GradientBoostingClassifier
            #('clf', AdaBoostClassifier(
            #    base_estimator=GradientBoostingClassifier(n_estimators=10000,max_depth=10,
            #       subsample=0.5,
            #       max_features='log2'),
            #    n_estimators=500,
            #    learning_rate=1,
            #    random_state=42,
            #    algorithm='SAMME',
            #))    
            
                
            ####### AdaBoost avec SVC
            #('clf', AdaBoostClassifier(
            #    svc(probability=True, kernel='linear'),
            #    n_estimators=10,
            #    learning_rate=1,
            #    random_state=42,
            #    algorithm='SAMME',
            #))
                
                
            ####### AdaBoost avec GaussianNB
            #('clf', AdaBoostClassifier(
            #    GaussianNB(),
            #  n_estimators=10,
            #  learning_rate=1,
            #  random_state=42,
            #  algorithm='SAMME',
            #))

        
            ####### AdaBoost avec SGDClassifier ou SVC
            #('clf', AdaBoostClassifier(
            #SGDClassifier(loss='log'),
            #n_estimators=10000,
            #learning_rate=1.6,
            #random_state=42,
            #algorithm='SAMME',
             #))
                

            ####### Bayesien 
            #('clf',  GaussianNB())
            
            ####### Multinomial NB
            #('clf', MultinomialNB(alpha=0.1, fit_prior=True, class_prior=None))
                
            ####### SGD 
            # ('clf',  SGDClassifier(loss='log')


                ])        



    def fit(self, X, y):
        self.clf.fit(X, y)

    def predict(self, X):
        return self.clf.predict(X)

    def predict_proba(self, X):
        return self.clf.predict_proba(X)

In [7]:
labels = data['CSPL_RECEIVED_CALLS'].unique()

In [8]:
print len( data['ASS_PARTNER'].unique())

16


In [9]:
y_df=data['CSPL_RECEIVED_CALLS'][:300000]

In [10]:
y_try=data['CSPL_RECEIVED_CALLS'][:3000000]
print len(y_try)

3000000


In [11]:
print len(y_df)

300000


In [19]:
X_df=data[  ['WEEK_END',  'DAY_WE_DS',  'TPER_TEAM',  'ASS_ASSIGNMENT',  'ASS_DIRECTORSHIP','ASS_PARTNER','year','month','day','hour','minute']][:300000]

In [13]:
print len(X_df)

300000


In [14]:

"""

from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

def train_test_model_clf(X_df, y_df, skf_is, FeatureExtractor, Classifier):
    
    train_is, test_is = skf_is
    
   
    
    # Feature extraction
    fe_clf = FeatureExtractor()
    fe_clf.fit(X_train_df, y_train_df)
    
    X_train_array_clf = fe_clf.transform(X_train_df)
    X_test_array_clf = fe_clf.transform(X_test_df)
    

    # Train
    clf = Classifier()
    clf.fit(X_train_array_clf, y_train_clf)
    # Test 
    y_proba_clf = clf.predict_proba(X_test_array_clf)                        
    y_pred_clf = labels[np.argmax(y_proba_clf, axis=1)]#on somme les 4 valeurs pour obtenir le label  
    #print y_proba_clf
    #print y_pred_clf
    #print np.argmax(y_proba_clf, axis=1)
    error = 1 - accuracy_score(y_test_clf, y_pred_clf)                       
    print('error = %s' % error)                                                                            
    print('classification report:\n %s' % classification_report(y_test_clf, y_pred_clf))
    print('confusion matrix:\n %s' % confusion_matrix(y_test_clf, y_pred_clf))


skf = ShuffleSplit(n_splits=2, test_size=0.2, random_state=57)  
skf_is = list(skf.split(X_df))[0]

train_test_model_clf(X_df, y_df, skf_is, FeatureExtractorClf, Classifier)
"""

"\n\nfrom sklearn.model_selection import ShuffleSplit\nfrom sklearn.metrics import classification_report, confusion_matrix, accuracy_score\n\ndef train_test_model_clf(X_df, y_df, skf_is, FeatureExtractor, Classifier):\n    \n    train_is, test_is = skf_is\n    \n   \n    \n    # Feature extraction\n    fe_clf = FeatureExtractor()\n    fe_clf.fit(X_train_df, y_train_df)\n    \n    X_train_array_clf = fe_clf.transform(X_train_df)\n    X_test_array_clf = fe_clf.transform(X_test_df)\n    \n\n    # Train\n    clf = Classifier()\n    clf.fit(X_train_array_clf, y_train_clf)\n    # Test \n    y_proba_clf = clf.predict_proba(X_test_array_clf)                        \n    y_pred_clf = labels[np.argmax(y_proba_clf, axis=1)]#on somme les 4 valeurs pour obtenir le label  \n    #print y_proba_clf\n    #print y_pred_clf\n    #print np.argmax(y_proba_clf, axis=1)\n    error = 1 - accuracy_score(y_test_clf, y_pred_clf)                       \n    print('error = %s' % error)                             

In [17]:
from sklearn.linear_model import SGDClassifier


clf3 = SGDClassifier()
clf2 = AdaBoostClassifier(
                   base_estimator=DecisionTreeClassifier(max_depth=8),
                   n_estimators=1000,
                   learning_rate=1.6,
                   random_state=42,
                   algorithm='SAMME',
               )
clf=Classifier()
if len(X_df) > 0:
    print "Training..."
    X = np.array(X_df)
    y = np.array(y_df).ravel()
    clf.fit(X,y)
print "End."



Training...
End.


In [14]:

print ("Loading the data submission...")
file = open('/Users/ozad/Desktop/AXA/submission.txt','r')
lines = [line.rstrip('\n') for line in file]
lines = lines[1:]
submission = []
for line in lines:
    tokens = line.split()
    date1 = tokens[0].split("-")
    date2 = tokens[1].split(":")
    date = datetime(int(date1[0]),int(date1[1]),int(date1[2]),int(date2[0]),int(date2[1]))
    ass_assignment = ' '.join(tokens[2:-1])
    submission.append((date, ass_assignment))
print "End."

Loading the data submission...
End.


In [22]:

m = len(submission)
print "Loading data for validation..."
posTest=6820
d_test = submission[len(submission)-1][0]
X_test = []
pos3=0
while pos3 < m and submission[pos3][0] < d_test:
    X_test.append([ data['ASS_ASSIGNMENT'][pos3] ])
    pos3 += 1
print "End."





Loading data for validation...
End.


In [23]:
print len(X_test)

82889


In [25]:
print "Prediction..."
if len(X_test) > 0:
    X_test = np.array(X_test)
    y_pred = clf.predict(X_test)
print pos3
print "End."

####### affichage 
'''
for i in range(pos3):
    print "%d-%d-%d %d:%d:00.000\t%s\t%d" % (submission[i][0].year,
        submission[i][0].month, submission[i][0].day,
        submission[i][0].hour, submission[i][0].minute, submission[i][1],
        y_pred[i])
'''



Prediction...


ValueError: Number of features of the model must match the input. Model n_features is 6 and input n_features is 1 

In [ ]:

def make_submission(test, prediction, filename='/Users/ozad/Desktop/axa_data_challenge-master/submission.txt'):
    """
    Create a submission file, 
    test: test dataset
    prediction: predicted values
    """
    with open(filename, 'w') as f:
        f.write('DATE\tASS_ASSIGNMENT\tprediction\n')
        submission_strings = test['DATE'] + '\t' + test['ASS_ASSIGNMENT'] + '\t'+ prediction.astype(str)
        for row in submission_strings:
            f.write(row + '\n') 

In [ ]:
make_submission(X_test,y_pred)